In [ ]:
'''
This component will spin up a MQTT client and subscribe to the topic to receive an image. 

Once Image is received it will compare it a known image and return whether
the received image matches the known image. 

TODO: How to receive client id in request.
TODO: Connect to MQTT server on TX2 to send the adjudication. 
TODO: Queue up and compare n images instead of just 1. Done.
TODO: Verify Secondary Auth.
TODO: Implement session tracking
TODO: Test

'''

#Libraries
from PIL import Image
import requests
from io import BytesIO
import ibm_boto3
from ibm_botocore.client import Config, ClientError
import face_recognition
import numpy as np

import time
import paho.mqtt.client as mqtt

#Helper Functions

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    print(client.subscribe("local/#"))

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    print("Message Received", time.time())
    msgRouter(msg)

# The logging callback
def on_log(client, userdata, level, buf):
    print('Log ', buf)
    
def msgRouter(msg):
    if msg.topic == FACES:
        queue('Vivek', msg)
    elif msg.topic == SECONDARY_AUTH_RESPONSE:
        secondary_auth('Vivek', msg)
    
def queue (user, msg):
    global image_queue
    image = face_recognition.load_image_file(BytesIO(msg.payload))
    if image_queue.get(user)==None:
        image_queue = {**image_queue, user:[image]}
        print('First image added for', user)
    else:
        images = image_queue.get(user)
        images.append(np.array(image))
        if len(images) >= batch_size:
            print('Sending', user, 'for adjudication.')
            response = adjudicate(user, image_queue.pop(user))
            print('Received', response)
            client.publish(response, qos=2, retain=False)
        else:
            image_queue = {**image_queue, user:images}
            print('Image appended for', user)

def adjudicate(user, images):
    
    url = known_images.get(user)
    response = requests.get(url)
    known_image = face_recognition.load_image_file(BytesIO(response.content))
    known_image_encoding = face_recognition.face_encodings(known_image)[0]
    
    matches = 0
    for unknown_image in images:
        #unknown_image = face_recognition.load_image_file(BytesIO(unknown_image_bytearr))
        unknown_image_encoding = face_recognition.face_encodings(unknown_image)[0]
        result = face_recognition.compare_faces([known_image_encoding], unknown_image_encoding)
        print('Comparing images for ', user)
        print('Result is', result)
        matches += result[0]

    match_ratio = matches/batch_size
    if match_ratio > give_money:
        return (GIVE_MONEY)
    elif match_ratio > secondary_authentication:
        return (SECONDARY_AUTH_REQUIRED)
    else:
        return (DECLINE)

def secondary_auth(user, msg):
    # Code to verify additional information. For now, just assume secondary authentication is successful
    client.publish(GIVE_MONEY, qos=2, retain=False)
    
# MQTT Standard code
client = mqtt.Client('Adjudicator')
client.on_connect = on_connect
client.on_message = on_message
client.on_log = on_log

MQTT_HOST = "127.0.0.1"
#MQTT_HOST = 'iot.eclipse.org'
MQTT_PORT = 1883

#Dictionary of known images. This will be replace by something more robust in production.
known_images = {
    'Vivek': 'https://s3.us-east.cloud-object-storage.appdomain.cloud/vivek/Vivek.png',
    'Mouli': 'https://s3.us-east.cloud-object-storage.appdomain.cloud/vivek/Mouli.png'
}

#Parameters
batch_size = 5 #Number of images to batch up
give_money = 0.7 #Threshold above which to dispense money
secondary_authentication = 0.2 # Threshold above which to ask for secondary authentication

#Incoming topics
FACES = 'local/faces'
START_TXN = 'local/start'
SECONDARY_AUTH_RESPONSE = 'local/response'

#Outgoing topics
GIVE_MONEY = 'adjudication/pass'
SECONDARY_AUTH_REQUIRED = 'adjudication/fail_face'
DECLINE = 'adjudication/terminate_info'


#Session tracker
session = {}

#Queue of items
image_queue = {}

print('Starting...')
print(image_queue)
client.connect(MQTT_HOST, MQTT_PORT, 60)
client.loop_forever()

Starting...
{}
Log  Sending CONNECT (u0, p0, wr0, wq0, wf0, c1, k60) client_id=b'Adjudicator'
Log  Received CONNACK (0, 0)
Connected with result code 0
Log  Sending SUBSCRIBE (d0, m1) [(b'local/#', 0)]
(0, 1)
Log  Received SUBACK
Log  Received PUBLISH (d0, q0, r0, m0), 'local/faces', ...  (334350 bytes)
Message Received 1563800825.723134
Log  Caught exception in on_message: queue() missing 1 required positional argument: 'msg'
Log  Received PUBLISH (d0, q0, r0, m0), 'local/faces', ...  (201023 bytes)
Message Received 1563800827.883801
Log  Caught exception in on_message: queue() missing 1 required positional argument: 'msg'
Log  Received PUBLISH (d0, q0, r0, m0), 'local/faces', ...  (762617 bytes)
Message Received 1563800829.935872
Log  Caught exception in on_message: queue() missing 1 required positional argument: 'msg'
Log  Received PUBLISH (d0, q0, r0, m0), 'local/faces', ...  (18485 bytes)
Message Received 1563800831.9545999
Log  Caught exception in on_message: queue() missing 1 re